### This is Natural Language Processing Project in which Natural Language Toolkit is used and base on Extraction Summarization

## Preparing the data. Here data is scraping from wikipedia

In [169]:
import bs4 as BeautifulSoup  #use to fetcha the articles text
import urllib.request        #use for connectiong to the page and retrieving the HTML

# Fetching the content from the URL
fetched_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Brahma_Kumaris')

article_read = fetched_data.read()

# Parsing the URL content and storing in a variable
article_parsed = BeautifulSoup.BeautifulSoup(article_read,'html.parser')

# Returning <p> tags
paragraphs = article_parsed.find_all('p')

article_content = ''

# Looping through the paragraphs and adding them to the variable
for p in paragraphs:  
    article_content += p.text

## Processing the data

In [170]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
def _create_dictionary_table(text_string) -> dict:
   
    # Removing stop words
    stop_words = set(stopwords.words("english"))
    
    words = word_tokenize(text_string)
    
    # Reducing words to their root form
    stem = PorterStemmer()
    
    # Creating dictionary for the word frequency table
    frequency_table = dict()
    for wd in words:
        wd = stem.stem(wd)
        if wd in stop_words:
            continue
        if wd in frequency_table:
            frequency_table[wd] += 1
        else:
            frequency_table[wd] = 1

    return frequency_table

In [171]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{'and', 'doesn', 'by', 'hasn', 'before', "should've", 'nor', 'for', 'yourself', "you'll", 'these', 'their', 'your', "wouldn't", 'o', 'aren', 'but', 'you', 'if', "didn't", 'too', "needn't", 'am', "shouldn't", 'ours', 'will', 'haven', 'it', "it's", 'as', 'm', 'once', 's', 'now', 'couldn', 'yourselves', 'each', 'into', "haven't", 'him', 'do', 'not', 'did', 'other', 'yours', 'its', "you'd", 'are', 'while', 'her', "hadn't", 'the', 'why', 'she', 'has', 'against', 'i', 'through', 'then', 'some', 'itself', "that'll", 'above', 'all', "she's", 'have', 'further', 'herself', 'whom', 'same', 'ourselves', 'myself', 'where', 'needn', 'very', 'about', 'was', 'at', "don't", 'can', 'didn', 'won', 'themselves', 'during', 'to', 'don', 'himself', 'ma', 'here', 'under', 'hadn', 'should', 'y', 'be', 'weren', "couldn't", 'again', 'who', 'having', 'own', 'isn', 'below', 'such', "isn't", 'in', 'after', 'that', 're', "won't", 'those', 'until', 'mightn', "you're", 'our', 'any', 'both', "aren't", 'mustn', 'does', 

In [172]:
frequency_table=_create_dictionary_table(article_content)


## Tokenizing the article into sentences

In [173]:
from nltk.tokenize import word_tokenize, sent_tokenize
sentences = sent_tokenize(article_content)

## Finding the weighted frequencies of the sentences

In [174]:
def _calculate_sentence_scores(sentences, frequency_table) -> dict:   

    # Algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence[:7] in sentence_weight:
                    sentence_weight[sentence[:7]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:7]] = frequency_table[word_weight]

        sentence_weight[sentence[:7]] = sentence_weight[sentence[:7]] /        sentence_wordcount_without_stop_words
      
    return sentence_weight

In [175]:
sentence_scores=_calculate_sentence_scores(sentences, frequency_table)

## Calculating the threshold of the sentences

In [176]:
def _calculate_average_score(sentence_weight) -> int:
   
    # Calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    # Getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score


In [177]:
threshold = _calculate_average_score(sentence_scores)

## Function to generate a summary from article

In [178]:
def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        if sentence[:7] in sentence_weight and sentence_weight[sentence[:7]] >= (threshold):
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

In [183]:
#producing the summary
summary_results = _get_article_summary(sentences, sentence_scores, 1.9* threshold)
print(summary_results)

 [45] A flagship slogan for the BKs has been When we change, the world changes.
